In [ ]:
library(ggplot2)
library(survival)
library(plyr)
library(dplyr)
library(stringr)
library(data.table)
library(tidyr)
library(corrplot)
library(Metrics)
library(caret)
library(dummies)
library(rpart)
library(rpart.plot)
library(e1071)
library(randomForest)
library(glmnet)
library(gbm)
library(Matrix)
library(iterators)
library(xgboost)
library(parallel)
library(parallelMap) 
library(caretEnsemble)
library(ensembleR)
library(caTools)
library(mlbench)
library(party)
library(ranger)
library(lars)
library(lightgbm)
library(pROC)
options(scipen = 999)

In [ ]:
df <- read.csv("../input/maindffinal20210912/maindffinal20210912.csv")

is.na(df)<-sapply(df, is.infinite)
df[is.na(df)]<-0

In [ ]:
df$NAME_CONTRACT_TYPE <- ifelse(df$NAME_CONTRACT_TYPE == "Cash loans",1,0)
df$CODE_GENDER <- ifelse(df$CODE_GENDER == "F",1,0)
df$FLAG_OWN_CAR <- ifelse(df$FLAG_OWN_CAR  == "N",1,0)
df$FLAG_OWN_REALTY <- ifelse(df$FLAG_OWN_CAR  == "N",1,0)

In [ ]:
df <-subset(df, select = -c(DAYS_EMPLOYED.1,X))

In [ ]:
train <- df %>% filter(data_split=="train")
test <- df %>% filter(data_split=="test")

In [ ]:
train1 <-subset(train, select = -c(data_split))
test1 <-subset(test, select = -c(data_split,TARGET))

In [ ]:

train1y <- subset(train1, select = c(TARGET))
train1x <-subset(train1,select = -c(TARGET))


train2x <- data.frame(lapply(train1x, as.numeric))
train2y <- data.frame(lapply(train1y, as.numeric))
test2 <- data.frame(lapply(test1, as.numeric))


train2xg <- as.matrix(train2x)
train2yg <- as.matrix(train2y)
testg <-as.matrix(test2)

modelx <- xgboost(data = train2xg, 
                  booster="gbtree",
                  objective = "binary:logistic",
                  eval_metric = "auc",
                  nthread = 4,
                  eta = 0.05,
                  nround = 2000,
                  max.depth = 6,
                  label = train2yg,
                  min_child_weight = 30,
                  subsample = 0.85,
                  colsample_bytree = 0.7,
                  colsample_bylevel = 0.632,
                  early_stopping_rounds = 97)

In [ ]:
predx <- predict(modelx,testg)

predxdf <- data.frame('SK_ID_CURR' = test1$SK_ID_CURR, 'Target' = predx)
write.csv(predxdf, file = "Submissionfile20210912.csv")